# Nginx

## 参考文献

* *深入理解Nginx，模块开发与架构解析*
* [nginx官方文档](http://nginx.org/en/docs/)
* *https://zhuanlan.zhihu.com/p/51251700*

# Nginx配置

## 编译安装Nginx

```shell
cd ~/github/nginx-1.4;
./auto/configure --prefix=/Users/hotbaby/github/nginx-1.4/ngin;

make -j8;
make isntall;
 
```

## Nginx进程间的关系

Nginx使用一个Master进程管理多个worker进程，一般情况下，worker进程的数量和服务器上CPU核心数相等。

* master进程不会对用户请求提供服务，只用于管理真正提供服务的worker进程，包括启动服务、停止服务、重载配置文件，平滑升级等；
* 多个worker进程处理互联网请求不但可以提供服务的健壮性，还可以利用现在SMP多核架构。

## Nginx配置优化

### 优化性能

Nginx worker进程数量

*  `worker_process number`
* `worker_cpu_affinity cpumask [cpumask ...]`

1. 多worker进程可以充分利用多核架构，但若worker进程的数量超过CPU核心数，那么会增大进程间切换带来的消耗，一般情况下worker进程与CPU核心数量相等
2. 绑定Nginx worker进程到指定的CPU内核

**为什么要绑定worker进程到指定的CPU内核？**

假定每个worker进程都非常繁忙，如果多个进程都会抢占同一个CPU，那么就会出现同步问题。反之，如果每个worker进程都独享一个CPU，就在内核调度策略上实现了完全的并发。

### 事件类配置

`accept_mutex  [on | off]`

`accept_mutex`可以让多个worker进程轮流、序列化与新的客户端建立TCP连接。当一个worker进程建立的连接数达到`worker_connections`配置最大连接数的7/8时，会大大地减少该worker进程试图建立TCP连接到机会，以此实现所有worker进程上处理客户端请求数尽量接近。

accept锁默认是打开的，如果关闭它，那么建立TCP连接到耗时会更短，但worker进程负载会非常不均衡。

选择事件模型

`use [select | poll | epoll | kqueue]`

每个worker的最大连接数

`worker_connections number`

## 配置静态Web服务器

### 虚拟主机与请求的分发

监听端口

`listen address:port default | default_server | backlog=num | deffered`

**`default` 将所在的server模块作为整个Web服务器默认的server模块。如果没有设置这个参数，那么将会以`nginx.conf`中找到的第一个server块作为默认的server块。**

> Nginx实践中，最好配置`default_server`，避免未匹配的请求打到不期望的的`server`上。

**为什么需要默认默认虚拟主机呢？**

当一个请求无法匹配配置文件中所有主机域时，就会选用默认的虚拟主机。

`backlog=num`

表示TCP中backlog队列的大小。默认为-1，表示不予设置。在TCP建立三次握手过程中，进程还没有开始处理监听的句柄，就会将这连接放在backlog队列中。如果backlog队列已满，还有新的的客户端试图通过三次握手建立TCP连接，就会失败。

`deferred`

设置该参数候，若用户发起建立连接请求，并且完成了TCP的三次握手，内核也不会为这次的连接调度worker进程来处理，只有用户真的发送请求数据，内存才会唤醒worker进程处理这个连接。这个参数使用大并发的情况下，它减轻了worker进程都负担。

主机名称

`server_name name [...]`

`server_name`后可以跟多个主机名称，例如`server_name www.bytehandler.com download.bytehandler.com`

在开始处理一个HTTP请求时，Nginx会取出header的Host，与每个server的`server_name`进行匹配，以此决定使用哪个`server`块来处理这个请求。存在多个server块配置，根据优先级匹配，server_name与Host匹配优先级如下：

1. 所有字符串完全匹配的`server_name`
2. 通配符在前面的`server_name`，例如：`*.bytehandler.com`
3. 通配符在后面的`server_name`，例如：`www.bytehandler.*`
4. 最后选择使用正则表达式才能匹配的`server_name`，例如：`~^\.bytehandler\.com`

> Nginx使用`server_name`配置针对特定的Host匹配不同的服务，以此实现虚拟主机功能。

`server_names_hash_bucket_size size;`

为了提高快速寻找相应的`server_name`的能力，Nginx使用散列表存储`server_name`，`server_names_hash_bucket_size`设置了每个散列桶占用内存的大小。

`server_names_hash_max_size size;`

`server_names_hash_max_size`会影响散列表的冲突率。`server_names_hash_max_size`越大，消耗的内存越大，散列key冲突的概率越低，检索的效率也会更快。

`location [=|~|~*|^~|@] uri/ {}`

`location`会尝试根据用户的URI来匹配上面的URI表达式。

* = 表示完全匹配
* ~ 表示匹配URI时是字母大小写敏感的
* ~* 表示匹配URI时，忽略字母的大小问题
* ^~ 表示匹配URI时，只需要前半部分与URI参数匹配即可

网络连接设置

keepalive超时时间

`keepalive_timeout time(默认单位：秒)`

一个keepalive连接在闲置超过一定的时间后（默认75秒），服务器和浏览都会去关闭这个连接。当然keepalive_timeout是用来约束Nginx服务器的，Nginx也会按照规范讲个这个时间传给浏览器。

`tcp_nodely on|off`

确定对`keepalive`连接是否使用`TCP_NODELY`选项。

## 用HTTP proxy module配置一个反向代理服务器

反向代理（reverse proxy）是指用户代理服务器接收Internet的连接请求，然后将请求转发给内部网络的上游服务器，并将上游服务器得到的结果返回给Internet上连接的客户端。

`upstream name {}`

```nginx
upstream backend {
    server backend1.bytehander.com;
    server backend2.bytehandler.com；
}
```

`server name [parametes]`

* `weigth=number`，设置这台服务器转发的权重，默认为1
* `max_fails=number`，该参数与`fail_timeout`配和使用，如果上游服务器转发失败次数超过number，则认为fail_timeout时间段内的上游服务器不可用。
* `fail_timeout=time`
* `down`，表示所有在上游服务器永久下线
* `backup`，在使用ip_hash配置时才有效，表示所在上游服务器只是备份服务器，只有所有非备份服务器上游服务器都失败后，才会向所在上游服务器转发请求

`ip_hash` 

在有些场景下，希望某一个用户的请求使用落在固定的一台服务器中。

`ip_hash`与`weight`配置不可同时使用。如果`upstream`集群中有一台上游服务器暂时不可用，不能直接删除配置，而是要配置`down`参数标识。

# Nginx基础架构

## worker进程是如何工作的

master进程是如何通知worker进程停止服务或更换日志文件？
master进程通过信号的方式与worker进程进行通信。worker进程会有一个ngx_signal_handler(int signo)方法处理信号。

ngx_terminate、ngx_quit、ngx_reopen都由ngx_signal_handler方法处理。
* 当收到QUIT信号，ngx_quit标志位置1，worker进程需要优雅的关闭进程
* 当收到TERM信号，ngx_terminate标志位置1，worker进程需要强制关闭进程
* 当收到USR1信号，ngx_reopen标志位置1，nginx需要重新打开文件（切换日志文件时）

<img src="https://bj.bcebos.com/ipic/Nginx-worker进程退出流程图.png" width="70%">

> `ngx_quit`置为1，关闭所有监听的句柄后，会将`ngx_eixt`标志位置1

## master进程是如何工作的

master进程不需要处理网络事件，通过管理worker等子进程实现重启服务、平滑升级、更换日志文件、配置文件生效等功能。

nginx master处理的信号

|信号|标志位|意义|
|---|-----|----|
|QUIT|`ngx_quit`|优雅的关闭进程|
|TERM或INT|`ngx_terminate`|强制关闭整个服务|
|USR1|`ngx_repopen`|重新打开服务中的所有文件|
|WINCH|`ngx_noaccept`|所有子进程不再接受新的连接|
|USR2|`ngx_change_binary`|平滑升级到新版本的Nginx程序|
|HUP|`ngx_reconfigure`|重新生效新的配置文件|
|CHLD|`ngx_reap`|子进程意外退出|

# 访问第三方服务

Nginx提供了两种异步的方式与第三方服务通信：upstream和subrequest。upstream可以保证在与第三方服务器交互时（包括三次握手建立TCP连接、发送请求、接收响应、四次挥手关闭连接）不会阻塞进程处理其他请求，Nginx仍然可以保持它的高性能。

## upstream的使用方式

Nginx的HTTP反向代理模块就是基于upstream方式实现的。

反向代理时先接收完客户端请求的HTTP包体后，才向上游服务器建立连接并转发请求的。

![](https://bj.bcebos.com/ipic/Nginx-Upstream执行的一般流程.PNG)

```c
struct ngx_http_upstream_s {
    //构造发往上游服务器的请求内容
    ngx_int_t (*create_request)(ngx_http_request_t *r);
    
    //收到上游服务器的响应就会回掉process_header方法。
    ngx_int_t (*process_header)(ngx_http_request_t *r);
    
    //销毁upstream请求时使用
    void (*finalize_request)(ngx_http_request_t *r, ngx_int_t rc);
    
    //5个可选的回调方法
    ngx_int_t (*input_filter_init)(void *data);
    ngx_int_t (*input_filter)(void *data, ssize_t bytes);
    ngx_int_t (*reinit_request)(ngx_http_request_t *r);
    void (*abort_request)(nx_http_request_r, *r);
    ngx_int_t (*rewrite_redirect)(ngx_http_request_t *r, ngx_table_elt_t *h, size_t prefix);
};
```

设置upstream限制性参数

```c
typedef struct {
    // 连接上游服务器的超时时间，单位毫秒
    ngx_msec_t connect_timeout;
    
    // 发送TCP包到上游服务器的超时时间，单位毫秒
    ngx_msec_t send_timeout;
    
    //接收上游TCP报文的超时时间，单位毫秒
    ngx_msec_t read_timeout;
} ngx_http_upstream_conf_t;
```

设置需要访问的第三方服务器的地址

```c
typedef struct {
    //地址个数
    ngx_uint_t naddsrs;
    
    //上游服务器地址
    struct sockaddr *sockaddr;
    socklen_t socklen;
    
} ngx_http_upstream_resolved_t;

```

## 回调方法的执行场景

### `create_request`回调方法

![](https://bj.bcebos.com/ipic/Nginx-create_request回调场景序列图.png)

1. Nginx主循环定期调用事件模块，以检查是否有网络事事件发生
2. 事件模块在接收HTTP请求后会调用HTTP框架来处理。假设解析完HTTP头后发现应该由mytest模块处理，这是会调用mytest模块的`ngx_http_mytest_handler`
3. 设置回调函数和第三方地址
4. 调用`ngx_http_upstream_init`方法启动upstream
5. upstream会检查文件缓存，如果缓存命中，直接返回结果
6. 回调mytest的`create_request`回调方法
7. mytest模块通过设置`r->upstream->request_bufs`发送请求到上游服务器
8. upstream模块将检查resovled成员，设置上游服务器的地址`r->upstream->peer`成员
9. 用无阻塞的TCP套接字建立连接
10.无论是否连接成功，负责建立连接的connect方法都会立刻返回
11. `ngx_http_upstream_init`返回
12. mytest模块的`ngx_http_mytest_handler`返回NGX_NONE
13. 当事件模块处理完网络事件后，将控制权交给Nginx主循环

### `reinit_request`回调方法

略

### `finalize_request`回调方法

略

### `process_header`回调方法

`process_header`是用于解析上游服务器返回的TCP响应头部，`process_header`可能会被多次调用。

* 如果返回值是`NGX_AGAIN`，会被再次调用
* 如果返回值是`NGX_OK`，则不会被再次调用

![](https://bj.bcebos.com/ipic/Nginx-process_header-回调场景的序列图.jpg)

1. Nginx主循环定期调用事件模块
2. 事件模块收到上游发来的响应时，会回调upsteam模块
3. upstream模块从套接字缓冲区中读取来自上游的TCP流
4. 读取响应存放到`r->upstream->buffer`指向的内存中
5. 调用mytest模块实现的`process_header`方法
6. `process_header`方法实际上是解析`r-upstream->buffer`缓冲区
7. 如果`process_header`返回`NGX_AGAIN`，表示还没有解析到完整的缓冲区头部，下次还会再次调用`process_header`e
8. 调用无阻塞套接字接口
9. 这时有可能套接字缓冲区已经为空
10. 读取上游响应事件处理完成，控制权还给事件模块
11. 事件模块处理完成后，交还控制权给Nginx主循环

### `rewrite_redirect`回调方法

略

### `input_filter_init`和`input_filter`回调

略

# 事件模块

Nginx是一个事件驱动的Web服务器。

## 事件处理框架概述

事件处理框架解决问题是如何收集、管理和分发事件。该事件主要是网络事件和定时器事件为主，而网络事件中又以TCP网络事件为主。

定时器事件简单独立，基于网络事件触发器实现，并不涉及操作系统内核。

`ngx_event_t`和`ngx_connection_t`是处理TCP连接的基础数据结构。

TODO

### Nginx事件的定义

TODO

### Nginx连接的定义

作为Web服务器，每个用户都对应一个TCP连接，为了及时处理这个连接，至少需要一个读事件和写事件，使得epoll可以有效的根据触发事件调度相应的模块读取或发送请求。Nginx使用`ngx_connection_t`来表示连接，这个连接是客户端发起的，Nginx被动接受的连接，称作**被动连接**。

同时，在有些请求的处理过程中，Nginx会试图主动向上游服务器建立连接，该连接称为**主动连接**，Nginx使用`ngx_peer_connection_t`数据结构表示主动连接。

#### 被动连接

未加修饰的“连接”都是指客户端发起的、服务器被动接收的连接，用`ngx_connection_t`数据结构表示。

TODO `ngx_connection_t`数据结构

#### 主动连接

作为Web服务器，Nginx也需要向其他服务器发起主动连接，用`ngx_peer_connection_t`数据结构表示。

TODO `ngx_peer_connection_t`数据结构。

#### `ngx_connection_t`连接池

参考`ngx_event_process_init`函数的实现。

Nginx在接收客户端的链接时，所使用的`ngx_connection_t`结构体都是在启动阶段就预分配好的，使用时从连接池中获取即可。

Nginx认为每一个连接至少需要一个读事件和一个写事件，有多少个连接就分配多少个读事件和写事件。

## `ngx_event_core_moudle`事件模块

`ngx_event_core_module`会创建连接池，决定使用那种事件驱动机制，初始化使用的时间模块。

![](https://bj.bcebos.com/ipic/Nginx-ngx_event_core_module事件模块启动时的工作流程.png)

## epoll事件驱动模块

### epoll的原理和用法

* `epoll_create`创建epoll对象
* `epoll_ctl`向epoll对象中添加socket
* `epoll_wait`收集发生事件

### 如何使用epoll

#### `epoll_create`系统调用

`int epoll_create(int size)`

`epoll_create`返回一个句柄，滞后epoll使用这个句柄标识，参数`size`标识大致事件的数目。

#### `epoll_ctl`系统调用

`int epoll_ctl(int epfd, int op, int fd, struct epoll_event* event);`

`epfd`是`epoll_create`返回的句柄

op的取值和含义

|op取值|含义|
|-----|-----|
|EPOLL_CTL_ADD|添加事件|
|EPOLL_CTL_MOD|修改事件|
|EPOLL_CTL_DEL|删除事件|

event的取值和含义

|event取值|含义|
|--------|----|
|EPOLLIN|对应连接有数据可读|
|EPOLLOUT|对应连接有数据可写|
|EPOLLRDHUP|表示TCP连接远端关闭或半关闭连接|
|EPOLLPRI|对应连接有紧急数据需要读|
|EPOLLERR|连接发生错误|
|EPOLLHUP|连接被挂起|
|EPOLLET|触发方式设置为边缘触发ET，系统默认是水平触发LT|
|EPOLLONESHOT|对这个事件只处理一次，下次需要处理时需重新加入|

#### `epoll_wait`系统调用

`int epoll_wait(int epfd, struct epoll_event* events, int maxevents, int timeout)`

* `epfd`是epoll文件描述符
* `events`是分配好的`epoll_events`结构体数组，epoll将把发生的事件复制到events数组中
* `maxevents`最大可以返回的事件数量
* `timeout`标识没有检测到事件发生时，最多的等待时间

## 定时器事件

略 TODO

## 事件驱动框架的处理流程

Nginx的post事件处理机制，post的意思是延后执行。Nginx设计了两个post队列，一个是被触发的监听连接事件构成的`ngx_posted_accept_events`队列，一个是普通读写事件构成的`ngx_posted_events`队列。

### `ngx_process_events_and_timers`流程

参考`ngx_process_events_and_times`函数

略

# FAQ

## Nginx如何解决“进群”问题？

什么是“进群”问题？

master进程开始监听Web端口，fork多个worker进程，这个子进程可能同时监听一个Web端口。某一个时刻所有worker进程都休眠且等待新连接的系统调用（如epoll_wait），这时一个用户向服务器发起了连接，内核在收到TCP的SYN包时，会激活所有的休眠的worker进程。此时，只有最新执行accept的worker进程可以成功创建连接，其他的worker进程都会accept失败。这些accept失败的子进程被内核唤醒时不必要的，且它们占用了本不需要的系统资源，引发了不必要的进程上下文切换，增加系统了开销。

**Nginx解决的方式很简单，它规定了同一个时刻只有唯一一个worker进程监听Web端口，这样就不会引发“进群”效应了。此时，新连接事件只能唤醒唯一正在监听的worker进程。**

Nginx如何限制某一个时刻只能有一个唯一worker进程监听Web端口呢？

参考`ngx_trylock_accept_mutex`方法的实现。只有获取`ngx_accept_mutex`锁成功后，才会将监听的fd注册到`epoll`ep对象中。

```c
ngx_int_t
ngx_trylock_accept_mutex(ngx_cycle_t *cycle)
{
    // 使用进程间同步锁，试图获取accept_mutex锁。
    // 返回1，表示成功拿到锁，返回0，表示获取锁失败
    // 获取锁定过程时非阻塞，如果被其他worker获取到锁，ngx_shmt_trylock方法立即返回
    if (ngx_shmtx_trylock(&ngx_accept_mutex))
    {
        // 成功获取到锁

        ngx_log_debug0(NGX_LOG_DEBUG_EVENT, cycle->log, 0,
                       "accept mutex locked");

        if (ngx_accept_mutex_held && ngx_accept_events == 0)
        {
            return NGX_OK;
        }

        // 将所有监听连接的读事件添加到当前的EPOLL的事件模块中
        if (ngx_enable_accept_events(cycle) == NGX_ERROR)
        {
            // 假如enable失败，释放 accept_mutex锁（正常情况不会进入该处理）
            ngx_shmtx_unlock(&ngx_accept_mutex);
            return NGX_ERROR;
        }

        ngx_accept_events = 0;
        ngx_accept_mutex_held = 1;

        return NGX_OK;
    }

    // 获取锁失败
    ngx_log_debug1(NGX_LOG_DEBUG_EVENT, cycle->log, 0,
                   "accept mutex lock failed: %ui", ngx_accept_mutex_held);

    if (ngx_accept_mutex_held)
    {
        // 将所有监听连接的读事件从事件驱动模块中移除
        if (ngx_disable_accept_events(cycle, 0) == NGX_ERROR)
        {
            return NGX_ERROR;
        }

        ngx_accept_mutex_held = 0;
    }

    return NGX_OK;
}

```

* `ngx_accept_mutex`实际上是进程间的同步锁

```c
static ngx_int_t
ngx_enable_accept_events(ngx_cycle_t *cycle)
{
    ngx_uint_t i;
    ngx_listening_t *ls;
    ngx_connection_t *c;

    ls = cycle->listening.elts;
    for (i = 0; i < cycle->listening.nelts; i++)
    {

        c = ls[i].connection;

        if (c == NULL || c->read->active)
        {
            continue;
        }

        if (ngx_add_event(c->read, NGX_READ_EVENT, 0) == NGX_ERROR)
        {
            return NGX_ERROR;
        }
    }

    return NGX_OK;
}
```

* 将fd注册到epoll中

## epoll的水平触发和边缘触发有什么区别？

epoll有两种工作模式：水平触发（LT）和边缘触发（ET）。默认情况下，epoll采用水平触发，这是可以处理阻塞和非阻塞套接字。

ET模式的效率比LT模式高，它仅只非阻塞套接字。ET与LT的区别在于，当一个新事件来到时，ET可以从`epoll_wait`调用获取这个事件，可是如果这次没有把这个事件对应的套接字缓冲区处理完，在套接字没有新的事件发生时，ET模式无法再次从`epoll_wait`调用获取这个事件；LT模式则相反，只要一个事件对应的套接字缓冲区有数据，总能从`epoll_wait`获取这个事件。

Nginx使用ET模式处理epoll事件。